In [0]:
!pip install tensorflow-gpu

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:

!pip install easycolab

In [0]:
import easycolab as ec
ec.mount()

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import glob
from tensorflow.keras.applications.vgg16 import preprocess_input

In [0]:
directory = glob.glob("/content/gdrive/My Drive/Image_retrival/images/*")
images = []
for path in directory:
  print(path)
  img = cv2.imread(path)
  img = np.array(img,np.float64)
  img = cv2.resize(img,(128,86))
  img = np.expand_dims(img,axis=0)
  img = preprocess_input(img)
  images.append(img)

In [0]:
from tensorflow import keras
def blocks_2(filters):
    seq = keras.Sequential()
    seq.add(keras.layers.Conv2D(filters,(3,3), activation='relu',padding='same'))
    seq.add(keras.layers.Conv2D(filters,(3,3),activation='relu',padding='same'))
    seq.add(keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    return seq
def blocks_3(filters):
    seq = keras.Sequential()
    seq.add(keras.layers.Conv2D(filters,(3,3),activation='relu',padding='same'))
    seq.add(keras.layers.Conv2D(filters,(3,3),activation='relu',padding='same'))
    seq.add(keras.layers.Conv2D(filters,(3,3),activation='relu',padding='same'))
    seq.add(keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    return seq

def VGG16():
    model = keras.Sequential()
    #block 1
    model.add(keras.layers.Conv2D(64,(3,3),input_shape = (86,128,3),padding = 'same',activation = 'relu')) 
    model.add(keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    #block 2
    model.add(blocks_2(128))
    model.add(blocks_3(256))
    model.add(blocks_3(512))
    model.add(blocks_3(512))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(4096,activation='relu'))
    model.add(keras.layers.Dense(100,activation='relu'))
    #model.add(keras.layers.Activation('softmax'))
#     sgd = keras.optimizers.SGD(lr=0.0005,decay=0,nesterov=True)
#     model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
    return model
net = VGG16()
net.summary()

In [0]:
query = cv2.imread('/content/gdrive/My Drive/Image_retrival/query_1.jpg')
query = np.array(query,np.float64)
query = cv2.resize(query,(128,86))
query_np = np.expand_dims(query,axis=0)
query_np = preprocess_input(query_np)

In [0]:
images = np.array(images).squeeze()
predict_query = net.predict(query_np)
predict_images_retrival =  net.predict(images)
print(predict_images_retrival.shape)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
pred_query  = np.reshape(predict_query,  (1, -1))
pred_images = np.reshape(predict_images_retrival, (predict_images_retrival.shape[0], -1))

In [0]:

def getKey(item):
    return item[0]
sim = cosine_similarity(pred_query, pred_images).squeeze()
image_response = [path for path in directory]
print(len(image_response))
tp = zip(sim.tolist(),image_response)
tp_s = sorted(tp, key=getKey, reverse=True)
img = cv2.imread('/content/gdrive/My Drive/Image_retrival/query_1.jpg')
#img = cv2.resize(img,(30,30))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.xlabel("Query")
plt.figure(figsize=(10,10))
for i in range(25):
    img = cv2.imread(tp_s[i][1])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(img)
    score = round(tp_s[i][0],5)
    plt.xlabel("score : " + str(score))
plt.show()
